# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,1.46,60,36,0.79,US,1733176930
1,1,albany,42.6001,-73.9662,-0.62,69,47,0.89,US,1733176692
2,2,namibe,-15.1961,12.1522,20.65,79,24,1.90,AO,1733176932
3,3,port-aux-francais,-49.3500,70.2167,6.40,97,100,8.50,TF,1733176934
4,4,vorgashor,67.5833,63.9500,-7.22,95,100,6.59,RU,1733176935


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 21)] # Narrow down to temperature between 18 to 26C
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Humidity"] < 80) & (ideal_weather_df["Humidity"] > 30)] # Narrow down to humidity between 30 to 70%
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10] # Narrow down to speed less than 10m/s.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0] # Narrow down to cloudiness = 0%

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,pisco,-13.7000,-76.2167,22.03,73,0,2.57,PE,1733176946
96,96,kone,-21.0595,164.8658,29.57,50,0,1.96,NC,1733177047
173,173,coahuayana de hidalgo,18.7000,-103.6583,29.50,59,0,2.12,MX,1733177144
179,179,doka,13.5167,35.7667,25.42,34,0,4.34,SD,1733177152
186,186,brisas de zicatela,15.8369,-97.0419,29.29,79,0,3.09,MX,1733177160
206,206,alice springs,-23.7000,133.8833,26.75,65,0,5.14,AU,1733177185
224,224,kailua-kona,19.6406,-155.9956,27.98,63,0,4.63,US,1733177099
510,510,gwadar,25.1216,62.3254,22.18,42,0,5.28,PK,1733177255


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
14,14,pisco,-13.7000,-76.2167,22.03,73,0,2.57,PE,1733176946,
96,96,kone,-21.0595,164.8658,29.57,50,0,1.96,NC,1733177047,
173,173,coahuayana de hidalgo,18.7000,-103.6583,29.50,59,0,2.12,MX,1733177144,
179,179,doka,13.5167,35.7667,25.42,34,0,4.34,SD,1733177152,
186,186,brisas de zicatela,15.8369,-97.0419,29.29,79,0,3.09,MX,1733177160,
206,206,alice springs,-23.7000,133.8833,26.75,65,0,5.14,AU,1733177185,
224,224,kailua-kona,19.6406,-155.9956,27.98,63,0,4.63,US,1733177099,
510,510,gwadar,25.1216,62.3254,22.18,42,0,5.28,PK,1733177255,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 100
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address =  requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
# Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
pisco - nearest hotel: La Portada
kone - nearest hotel: Pacifik Appartels
coahuayana de hidalgo - nearest hotel: No hotel found
doka - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: Casa de Olas
alice springs - nearest hotel: Aurora Alice Springs
kailua-kona - nearest hotel: PACIFIC 19 Kona
gwadar - nearest hotel: Sadaf Resorts


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
14,14,pisco,-13.7000,-76.2167,22.03,73,0,2.57,PE,1733176946,La Portada
96,96,kone,-21.0595,164.8658,29.57,50,0,1.96,NC,1733177047,Pacifik Appartels
173,173,coahuayana de hidalgo,18.7000,-103.6583,29.50,59,0,2.12,MX,1733177144,No hotel found
179,179,doka,13.5167,35.7667,25.42,34,0,4.34,SD,1733177152,No hotel found
186,186,brisas de zicatela,15.8369,-97.0419,29.29,79,0,3.09,MX,1733177160,Casa de Olas
206,206,alice springs,-23.7000,133.8833,26.75,65,0,5.14,AU,1733177185,Aurora Alice Springs
224,224,kailua-kona,19.6406,-155.9956,27.98,63,0,4.63,US,1733177099,PACIFIC 19 Kona
510,510,gwadar,25.1216,62.3254,22.18,42,0,5.28,PK,1733177255,Sadaf Resorts


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "Hotel Name",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,Country)